In [2]:
import networkx as nx
# import urllib -- Did not work on my linux machine - using requests instead
import re
import tqdm
import collections
import requests
#import community.community_louvain as cl
import json
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [6]:

# to create the network I will create a simple function that gets references from wiki article

def get_references(text, pattern=r'\[\[(.*?)\]\]'):
    if text is None:
        return None
    matches = re.findall(pattern, text)

    matches_more = []
    for match in matches:
        x = match.split('|')
        for m in x:
            matches_more.append(m)

    return matches_more

#The function above uses regex to gather everything inside double square brackets `[[]]`. We know that there might be more than one option there so after that we split everything on `|` mark.  



In [7]:

# Creating function to load up the rapper page

def get_wikipage(wiki_name):
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    title = f"titles={wiki_name.replace(' ', '_')}" # replacing space with _
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json" # we want have it in json format

    try:
        query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)


        wikitext = requests.get(query).content.decode('utf-8') # decoding to utf

        d = json.loads(wikitext)

        # now we want the actual latest article from wikipedia,
        # so we have to access this information within dictionaries
        page = d['query']['pages']
        page_id = list(page.keys())[0]
        page_title = page[page_id]['title']
        content = page[page_id]['revisions'][0]['*']
    except:
        return None, None

    return page_title, content


In [8]:
page_title, content = get_wikipage('List_of_music_genres_and_styles')

In [9]:
content

"{{Short description|Wikipedia list article}}\n{{Popular music}}\n\nThis is a '''list of music genres and styles'''. [[Music]] can be described in terms of many [[music genre|genres]] and styles. Classifications are often arbitrary, and may be disputed and closely related forms often overlap. Larger genres and styles comprise more specific sub-categories.\n\n==Art & Classical==\n{{see also|List of classical and art music traditions}}\n* [[Andalusian classical music]]\n* [[Indian classical music]]\n* [[Korean court music]]\n* [[Persian classical music]]\n* [[Western classical music]]\n** [[Early music]] \n*** [[Medieval music]] (500–1400) \n**** [[Ars antiqua]] (1170–1310)\n**** [[Ars nova]] (1310–1377)\n**** [[Ars subtilior]] (1360–1420)\n****[[Bardcore]]\n****[[Neo-Medieval music]]\n****[[Tavernwave]]\n*** [[Renaissance music]] (1400–1600)\n***[[Baroque music]] (1600–1750)\n*** [[Galant music]] (1720–1770)\n***[[Classical period (music)|Classical period]] (1750–1820)\n***[[Romantic mu

In [10]:
music_list = get_references(content)

In [11]:
better_music_list = copy.copy(music_list)

In [12]:
for x in better_music_list:
    if x + ' (music)' in better_music_list:
        better_music_list.remove(x)
    elif x + ' music' in better_music_list:
        better_music_list.remove(x)
    elif x + ' in music' in better_music_list:
        better_music_list.remove(x)
    elif x + ' (genre)' in better_music_list:
        better_music_list.remove(x)



In [13]:
with open ('music_genres_new.txt', 'w', encoding='utf-8') as f:
    for x in better_music_list:
        f.writelines(x + '\n')

In [14]:
# I then manually go through music genres and manually remove some of the values

In [15]:
with open ('music_genres.txt', 'w', encoding='utf-8') as f:
    for x in better_music_list:
        f.writelines(x + '\n')

In [16]:
_,wikipage_test = get_wikipage(music_list[5]) #cultural_origins[^|]*
test_text = get_references(wikipage_test, pattern=r'cultural_origins[^|]*')
print(test_text)

['cultural_origins  = Early 600s – Mid-1400s, Korea\n']


In [17]:
def extract_origins(text):
    '''
    input is expected to be the raw output from get_references()
    '''
    start, end = 0,0
    matches_num = re.findall(r'\d+', text[0])
    
    if len(matches_num) == 2:
        # Contains two years, use these as start and end point
        start = int(matches_num[0])
        end = int(matches_num[1])
        
    elif len(matches_num) == 1:
        # Contains one year, likely decade, use start and end of this decade
        prefix = matches_num[0][:-1]
        start = int(prefix+"0")
        end = int(prefix+"9")
        
    elif len(matches_num) == 0:
        # No year given, write as unknown
        start, end = -1, -1
        
    else:
        raise Exception(f"UnhandledOriginException: the text \"{text[0]}\" contains more years than 2")
    #print(text[0])
    print(re.findall(r'\b[^[\]\s]+\b$', text[0])[-1])
    place = re.findall(r'\b[^[\]\s]+\b$', text[0])[-1]
    
    return start, end, place
print(extract_origins(['cultural_origins  = Early 600s – Mid-1400s, Korea\n']))
print(extract_origins(['cultural_origins  = Early 600s – Mid-, Korea\n']))
print(extract_origins(['cultural_origins  = Early  – Mid-, Korea\n']))
#print(extract_origins(['cultural_origins  = Early 600s – Mid-1400s sub 30s, Korea\n']))

Korea
(600, 1400, 'Korea')
Korea
(600, 609, 'Korea')
Korea
(-1, -1, 'Korea')


In [18]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

def extract_origins_ner(text):

    tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
    model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
    
    nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    result = nlp(text)
    
    dates = []
    places = []
    
    for item in result[0]:
        if item['entity_group'] == 'DATE':
            dates.append(item['word'])
        elif item['entity_group'] == 'LOC':
            places.append(item['word'])
        
    return dates, places
    
    
print(extract_origins_ner(['cultural_origins  = Early 600s – Mid-1400s, Korea\n']))
#print(extract_origins_ner(['cultural_origins  = Early 600s – Mid-, Korea\n']))
#print(extract_origins_ner(['cultural_origins  = Early  – Mid-, Korea\n']))

C:\Users\tobia\anaconda3\envs\nerf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(['600s', 'Mid-1400s'], ['Korea'])


In [15]:
!pip install sentencepiece

In [19]:
_, wikipage_test = get_wikipage(music_list[5]) #cultural_origins[^|]*
print(get_references(wikipage_test, pattern=r'cultural_origins[^\n]*'))

origin_dict = {}

for genre in better_music_list:
    _,wikipage_test = get_wikipage(genre) #cultural_origins[^|]*
    test_text = get_references(wikipage_test, pattern=r'cultural_origins[^\n]*')
    if test_text != [] and test_text is not None:
        try:
            origin_dict[genre] = extract_origins_ner(test_text)
            print("succeeded with", genre)
            continue
        except:
            print("failed with", genre)
    else:
        print(f'failed to extract origins for {genre}')

['cultural_origins  = Early 600s – Mid-1400s, Korea']
failed to extract origins for Music
failed to extract origins for music genre
failed to extract origins for genres
failed to extract origins for Andalusian classical music
failed to extract origins for Indian classical music
succeeded with Korean court music
failed to extract origins for Persian classical music
failed to extract origins for Western classical music
failed to extract origins for Early music
failed to extract origins for Medieval music
failed to extract origins for Ars antiqua
failed to extract origins for Ars nova
failed to extract origins for Ars subtilior
succeeded with Bardcore
failed to extract origins for Neo-Medieval music
failed to extract origins for Tavernwave
failed to extract origins for Renaissance music
failed to extract origins for Baroque music
failed to extract origins for Galant music
failed to extract origins for Classical period (music)
failed to extract origins for Romantic music
failed to extract 

In [21]:
with open('origin_dict.txt', 'w') as convert_file: 
     convert_file.write(json.dumps(origin_dict))

In [20]:
origin_dict

{'Korean court music': (['600s', 'Mid-1400s'], ['Korea']),
 'Bardcore': (['2010s'], []),
 'Minimal music': (['1960s'], ['United States']),
 'Danger music': (['1960s'], []),
 'Drone music': (['1960s'], ['United States']),
 'Industrial music': (['-1970s'],
  ['United Kingdom', 'United States', 'Chicago]]', 'Germany']),
 'Instrumental': ([], []),
 'African blues': ([], ['ica']),
 'Blues rock': (['mid-1960s'], ['US', 'UK']),
 'British blues': ([], ['Mid-twentieth-', 'United Kingdom']),
 'Canadian blues': ([], ['Canada']),
 'Chicago blues': ([], ['Chicago', 'U.S.']),
 'Classic female blues': ([], ['Southern United States']),
 'Delta blues': ([], ['[Mississippi]]', 'U.S.']),
 'Desert blues': (['From 1980s to 1990s'],
  ['[[Tuareg]', '[North Africa', '[West Africa', '[Sahara]']),
 'Electric blues': (['1930s'], ['United States']),
 'Gospel blues': ([], []),
 'Hill country blues': ([], ['[Mississippi]]', 'U.S.']),
 'Jump blues': (['1930s'], ['U.S.']),
 'Louisiana blues': (['1940s–1950s'],
  ['[

In [4]:
with open('origin_dict.txt') as f: 
    data = f.read() 
      
# reconstructing the data as a dictionary 
js = json.loads(data)
len(js.keys())

711

In [19]:
len(better_music_list)

1555